# animate

> Animate the graph of recursive functions calls

In [ ]:
#| default_exp animate

In [ ]:
#| export
from recursion_visualizer.node import Node 
from recursion_visualizer.graph import get_graph

In [ ]:
#| export
import networkx as nx
import plotly.graph_objects as go

from typing import List, Dict
from plotly.subplots import make_subplots

## Node and Edge Coordinates

In [ ]:
#| export
def get_node_and_edge_coordinates(
  DG, # `networkx` graph of recursive function calls
  ) -> List[List[int]]: # list of x and y coordinates for placing nodes and edges on 2D plot
  """Given a map of edges to labels, create a `networkx` graph and use its
  layout function to return a list of x and y coordinates for placing nodes and edges on
  a 2D plot."""

  # get the (x, y) coordinates of each node in the graph
  coordinates = nx.drawing.nx_agraph.graphviz_layout(DG, prog="dot", root=0)

  # record the (x, y) coordinates of each node
  node_x, node_y = [], []
  for x, y in coordinates.values():
    node_x.append(x)
    node_y.append(y)

  # record the (x, y) coordinates of each edge
  edge_x, edge_y = [], []
  for edge in DG.edges():
        x0, y0 = coordinates[edge[0]]
        x1, y1 = coordinates[edge[1]]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

  return node_x, node_y, edge_x, edge_y

## Text

In [ ]:
#| export
def get_node_text(
   nodes: Dict[int, Node], # map of node ids to nodes
   func_name: str, # name of the recursive function
   ) -> List[List[str]]: # text(s) to be displayed on each node
   """Return the text(s) to be displayed on each node. Specifically, return
  
   1. `max_node_length`: the length of the longest node annotation text; used to
      determine the size of the node (in pixels)
   2. `node_annotations`: the text that appears in each node; this is set to be the
      input to the recursive function
   3. `hovertext`: the text that appears when the user hovers over a node in
      the graph; this is set to the input+output to the recursive function and the
      discovery+finish times of the recursive function.
   """
  
   hovertext, node_annotations = [], []
   max_node_length = 0

   for node in nodes.values():
      function_input = ','.join(list(map(str, node.input)))
      text = '{}({})={}'.format(func_name, function_input, node.output)
      text += '<br>discover: {}<br>finish: {}'.format(node.discovered, node.finish)

      node_annotations.append(function_input)
      max_node_length = max(max_node_length, len(function_input))
      hovertext.append(text)
   return max_node_length, node_annotations, hovertext

In [ ]:
#| export
def get_edge_text(
      node_x: List[int], # x coordinates of nodes
      node_y: List[int], # y coordinates of nodes
      edge_to_label: Dict[tuple, str], # map of edges to labels (edge is a tuple of two node ids)
      ) -> List[str]: # text(s) to be displayed on each edge
      """Return the text(s) to be displayed on each edge. This is optional and
      by default will display an empty string along each edge. Specifically, return
      
      1. `edge_text`: the text that appears along each edge; this is set to be
         `edge_label`, a keyword found in the decorated recursive function.
      """

      # initial values
      buffer = 5
      edge_annotations = []

      for i, ((node1, node2), label) in enumerate(edge_to_label.items()):
            x1, y1 = node_x[node1], node_y[node1]
            x2, y2 = node_x[node2], node_y[node2]
            edge_annotations.append(
                  dict(text=label,
                        x=(x1+x2)/2 + buffer, y=(y1+y2)/2 + buffer,
                        xref='x1', yref='y1',
                        font=dict(color='rgb(0,0,0)', size=10),
                        showarrow=False)
                  )
      return edge_annotations    

In [ ]:
#| export
def get_link_annotation(
    ) -> List[dict]: # text that links to this library's GitHub repo
    """Return the (formatted) text that links to this library's GitHub repo."""
    
    return [dict(
        text="Made with <a href='https://github.com/ez2rok/recursion-visualizer'>Recursion Visualizer</a>",
        showarrow=False,
        x=0.005, y=-0.002,
        xref="paper", yref="paper",
        )
    ]

In [ ]:
#| export
def get_title(
    nodes: Dict[int, Node], # map of node ids to nodes
    func_name: str, # name of the recursive function
    ) -> str: # title of the graph
    """Return the (formatted) title of the graph"""
    
    function_input = ','.join(list(map(str, nodes[0].input)))
    title_text = 'Recursive Tree: {}({})'.format(func_name, function_input)
    return {
        'text': title_text,
        'y':0.95,
        'x':0.02,
        'xanchor': 'left',
        'yanchor': 'top'
        }

## Graphics

In [ ]:
#| export
def get_slider() -> dict: # slider to control the animation
  """Return the slider that allows the user to control the animation"""
    
  return {
      "active": 0,
      "yanchor": "top",
      "xanchor": "left",
      "currentvalue": {
          "font": {"size": 20},
          "prefix": "Time:",
          "visible": True,
          "xanchor": "right"
      },
      "transition": {"duration": 300, "easing": "cubic-in-out"},
      "pad": {"b": 5, "t": 5},
      "len": 0.9,
      "x": 0.1,
      "y": 0,
      "steps": []
  }

In [ ]:
#| export
def update_slider(
  slider: dict, # slider that allows the user to control the animation 
  time: int, # current time
  ) -> dict: # updated slider
  """Update the slider with the current time"""
    
  slider_step = {
      "args": [[time],
               {"frame": {"duration": 300, "redraw": False},
                "mode": "immediate",
                "transition": {"duration": 300}}
               ],
               "label": time,
                "method": "animate"
                }
  slider["steps"].append(slider_step)

In [ ]:
#| export
def get_play_pause_buttons() -> List[dict]: # play/pause buttons
    """Return the play/pause buttons
    
    **References**:
    
    1. Heavily inspired by [Plotly animation
       tutorial](https://plotly.com/python/animations/#using-a-slider-and-buttons)
    """
    return [
    {"buttons": 
     [{"args": [None, {"frame": {"duration": 500, "redraw": False},
                       "fromcurrent": True, 
                       "transition": {"duration": 300, "easing": "quadratic-in-out"}
                       }
                ],
       "label": "Play",
       "method": "animate"
       },
      {"args": [[None], {"frame": {"duration": 0, "redraw": False},
                         "mode": "immediate",
                         "transition": {"duration": 0}
                         }
                ],
       "label": "Pause",
       "method": "animate"
       }
      ],
     "direction": "up",
     "pad": {"r": 20, "t": 15},
     "showactive": True,
     "type": "buttons",
     "x": 0.1,
     "xanchor": "right",
     "y": 0,
     "yanchor": "top"
     }
     ]

In [ ]:
#| export
def get_axis_settings() -> dict: # axis settings
  """Return the axis settings"""
  
  return dict(showline=False,
              zeroline=False,
              showgrid=False,
              showticklabels=False
              )

## Generate Animation

In [ ]:
#| export
def animate(history,nodes,
            func_name, 
            DG, 
            edge_to_label
            ):
  
  # get node and edge coordinates
  node_x, node_y, edge_x, edge_y = get_node_and_edge_coordinates(DG)
  
  # get text to be displayed on the nodes, edges, and figure
  max_node_length, node_annotations, hovertext = get_node_text(nodes, func_name)
  edge_annotations = get_edge_text(node_x, node_y, edge_to_label)
  link_annotation = get_link_annotation()
  title = get_title(nodes, func_name)
  
  # get graphics
  slider = get_slider()
  axis_settings = get_axis_settings()
  buttons = get_play_pause_buttons()

  # keep track of the visit status of each node at each time step:
  #   0 -> unvisited
  #   1 -> visiting
  #   2 -> visited
  # This is used to color the node circles and the node text.
  visit = [0] * len(nodes)
  visit_to_text_color =  {0: 'rgb(0,0,0)', 1: 'rgb(255, 255, 255)', 2: 'rgb(255, 255, 255)'}
  node_text_color = [visit_to_text_color[visit[id_]] for id_ in nodes.keys()]

  # get frames
  frames = []
  fig = make_subplots(specs=[[{"secondary_y": True}]])
  for time, node_id in enumerate([None] + history):
    
    # update visit status, node text color, and circle color
    # Note: circle color is determined by `visit`` so by updating `visit`, we've updated circle color
    if node_id != None:
      visit[node_id] += 1
      node_text_color[node_id] = visit_to_text_color[visit[node_id]]

    # plot nodes
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hovertext=hovertext,
        hoverinfo='text',
        text=node_annotations,
        #textfont={'color': node_annotation_colors[time]},
        textfont={'color': node_text_color},
        marker={'line': dict(color='rgb(50,50,50)', width=1),
                'color': visit,
                'cmid': 1,
                'colorscale': [(0.00, '#F7FBFF'), (0.33, '#F7FBFF'),
                               (0.33, '#6AAED6'), (0.66, '#6AAED6'),
                               (0.66, '#0A306B'), (1.00, '#0A306B')
                               ], # colors taken from px.colors.sequential.Blues
                'size': 10 + 7*max_node_length},
        showlegend=False,
        ids=list(nodes.keys())
        )
    
    # plot edges
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines',
        showlegend=False,
        textposition="bottom right"
        )
    
    # updates frames and slider
    frame = go.Frame(data=[node_trace, edge_trace], name=str(time))
    frames.append(frame)
    update_slider(slider, time)

  # create layout
  layout = go.Layout(
      title=title,
      xaxis=axis_settings,
      yaxis=axis_settings,
      annotations=link_annotation+edge_annotations,
      margin=dict(b=0,l=5,r=5,t=5),
      updatemenus=buttons,
      sliders=[slider]
      )

  fig = go.Figure(data=frames[0]['data'], layout=layout, frames=frames[1:])
  return fig

# Memoize Class

In [ ]:
#| export
class Memoize:
  """A class that provides a decorator for visualizing recursion trees and caching results."""

  def __init__(self,
               verbose: bool = False, # if true, print all nodes
               animate: bool = True, # if true, create an animation of the recursion tree
               save: bool = False, # if true, save the animation to a html file
               path: str ='', # path to save the animation to
               ): 
    
    self.verbose = verbose
    self.animate = animate
    self.save = save
    self.path = path
    self._reset()

  def _reset(self):
    """
    self.nodes = preorder traversal of nodes
    self.history = element i was discovered or finished at time i
    self.pos = position of vertices in animate
    """
    self.nodes, self.node_to_edge_labels, self.history = {}, {}, []
    self.id_, self.time, self.depth = 0, 0, 0
    self.cache = {}
    self.func_name = ''

  def _animate(self, history, nodes, node_to_edge_labels, func_name):
    
    DG, edge_to_label = get_graph(history, nodes, node_to_edge_labels)
    fig = animate(history, nodes, func_name, DG, edge_to_label)
    # HTML(fig.to_html())
    fig.show()
    return fig



    # # create recursion tree animation
    # fig = animate(history, nodes, func_name)
    # fig.show()

    # # save figure
    # if self.save:
    #   if self.path == '':
    #     input = ','.join(list(map(str, nodes[0]['input'])))
    #     self.path = './{}_{}.html'.format(func_name, input)
    #   fig.write_html(self.path)

  def __call__(self, 
               func: callable # function to be visualized or cached via decorator
               ):
    """A custom `__call__` function records the id, function input, function output, depth, discovery time, 
    and finish time in a node each time the function is called. After all function calls are made, `__call__`
    will animate the recursion tree. This is the main workhorse of the `RecursionVisualizer` class.
    
    At a high-level, the `__call__` function looks something like:
    
    ```
    def __call__(self, func):
      def memoized_func(*args, **kwargs):
        # record discovery time, function input, and depth
        node.discovery = time
        node.input = args
        node.depth = depth
        
        # if node not in cache, compute and cache result
        if node not in self.cache:
          self.cache[args] = func(*args, **kwargs)
          
        # record finish time and function output
        node.output = self.cache[args]
        node.finish = time
        
        if depth == 0:
          animate()
        
      return memoized_func
    ```
    """

    def memoized_func(*args, **kwargs):
      if self.depth == 0:
        self._reset()

      # record node's depth, discovery time, and input arguments
      id_ = len(self.nodes)
      node = Node(id_=id_, input=args, depth=self.depth, discovered=self.time)
      self.history.append(node.id_)
      self.nodes[node.id_] = node
      self.time += 1

      # update depth and call the function `func`
      self.depth += 1
      # if args not in self.cache:
      self.cache[args] = func(*args, **kwargs)
      self.depth -= 1

      # record node's output, finish time, history, and edge_label
      self.nodes[id_].output = self.cache[args]
      self.nodes[id_].finish = self.time
      
      self.node_to_edge_labels[id_] = kwargs['edge_label'] if kwargs and 'edge_label' in kwargs else ''
      self.history.append(node.id_)
      self.time += 1

      if self.verbose:
        print(node)

      # animate after done traversing through the entire tree
      if self.animate and self.depth == 0:
        fig = self._animate(self.history, self.nodes, self.node_to_edge_labels, func.__name__)
        return fig, self.cache[args]

      return self.cache[args]
    return memoized_func

In [ ]:
#F7FBFF Light blue
#6AAED6 Medium Blue
#0A306B Dark Blueh

In [ ]:
#| export
import plotly.io as pio
from IPython.display import display, HTML
from IPython import get_ipython

In [ ]:
#| export
def on_colab():   # cf https://stackoverflow.com/questions/53581278/test-if-notebook-is-running-on-google-colab
    """Returns true if code is being executed on Colab, false otherwise"""
    try:
        return 'google.colab' in str(get_ipython())
    except NameError:    # no get_ipython, so definitely not on Colab
        return False 

if not on_colab():  # Nick Burrus' code for normal-Juptyer use with plotly:
    pio.renderers.default = 'notebook_connected'
    js = '<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js" integrity="sha512-c3Nl8+7g4LMSTdrm621y7kf9v3SDPnhxLNhcjFJbKECVnmZHTdo+IRO05sNLTH/D3vA6u1X32ehoLC7WFVdheg==" crossorigin="anonymous"></script>'
    display(HTML(js))

In [ ]:
# on_colab()

# Fibonacci Example

In [ ]:
@Memoize()
def fib(n, edge_label=''):
  if n <= 2: return 1

  val = 0
  for decrement in [1, 2]:
    val += fib(n-decrement, edge_label='-{}'.format(decrement))
  return val

In [ ]:
#|output: true
fig, result = fib(5)

# Partition Equal Subset Sum

In [ ]:
# def canPartition(nums):
    
#     total = sum(nums)
#     cache = {}
    
#     @Memoize()
#     def f(i, sum_, edge_label=''):
#         if i == len(nums): # base case
#             cache[(i, sum_)] = total - sum_ == sum_  
#             return cache[(i, sum_)]
#         elif (i, sum_) in cache: # already computed
#             return cache[(i, sum_)]
#         else: # main case
#             cache[(i, sum_)] = f(i+1, sum_ + nums[i], edge_label=edge_label+str(nums[i]) + ',') \
#              or f(i+1, sum_, edge_label=edge_label)
#         return cache[(i, sum_)]
    
#     return f(0, 0)

In [ ]:
# nums = [2,3,4,5,6]
# canPartition(nums)

# Coin Change Example

In [ ]:
# def coin_change(coins, amount):

#   @Memoize(save=True)
#   def change(i, edge_label=''):
#     if i == 0 or i < 0: # base case
#      return 0 if i == 0 else float('inf')
        
#     result = float('inf')
#     for c in coins:
#       if i - c >= 0:
#         result = min(result, change(i-c, edge_label='{} coin'.format(c)) + 1)
#     return result
        
#   result = change(amount)
#   return -1 if result == float('inf') else result

In [ ]:
# coins = [1, 3, 5]
# amount = 7
# coin_change(coins, amount)

# Merge Sort

In [ ]:
# def mergesort_wrapper(nums):

#   def merge(lo, mid, hi):
#     L, R = nums[lo:mid+1] + [float('inf')], nums[mid+1:hi+1] + [float('inf')]
#     i, j = 0, 0
#     for k in range(lo, hi+1):
#       if L[i] <= R[j]:
#         nums[k] = L[i]
#         i += 1
#       else:
#         nums[k] = R[j]
#         j += 1

#   @Memoize()
#   def mergesort(lo, hi, edge_label=''):

#     if lo < hi:
#       mid = lo + (hi-lo) // 2
#       mergesort(lo, mid, edge_label='nums={}'.format(nums[lo:mid+1]))
#       mergesort(mid+1, hi, edge_label='nums={}'.format(nums[mid+1:hi+1]))
#       merge(lo, mid, hi)
#       return nums[lo:hi+1]

#   mergesort(0, len(nums)-1)  
#   return nums

In [ ]:
# nums = [3, 1, 9, 4]
# result = mergesort_wrapper(nums)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

/Users/eitanturok/mambaforge/envs/recursion-visualizer/lib/python3.10/site-packages/nbdev/export.py:54: UserWarning:

Notebook '/Users/eitanturok/Desktop/projects/recursion-visualizer/02_animate.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.

